## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
type_count = application_df["APPLICATION_TYPE"].value_counts()
type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(type_count[type_count < 528].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df["CLASSIFICATION"].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
filtered_classification_count = classification_count.loc[classification_count > 1]
filtered_classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_count[classification_count < 1883].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

**Optimization Model 1**

In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_39 (Dense)            (None, 80)                3520      
                                                                 
 dense_40 (Dense)            (None, 30)                2430      
                                                                 
 dense_41 (Dense)            (None, 10)                310       
                                                                 
 dense_42 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6271 (24.50 KB)
Trainable params: 6271 (24.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [41]:
# Train the model
nn2_fit = nn2.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5704 - accuracy: 0.7223
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5531 - accuracy: 0.7305
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5511 - accuracy: 0.7315
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5488 - accuracy: 0.7324
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5489 - accuracy: 0.7321
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7311
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7331
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7329
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5463 - accuracy: 0.7341
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5453 - accura

In [43]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5556 - accuracy: 0.7301 - 448ms/epoch - 2ms/step
Loss: 0.5555810332298279, Accuracy: 0.7301457524299622


**Optimization Model 2**

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 5
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

#Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                3520      
                                                                 
 dense_13 (Dense)            (None, 30)                2430      
                                                                 
 dense_14 (Dense)            (None, 5)                 155       
                                                                 
 dense_15 (Dense)            (None, 1)                 6         
                                                                 
Total params: 6111 (23.87 KB)
Trainable params: 6111 (23.87 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
nn3_fit = nn3.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6020 - accuracy: 0.7048
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5625 - accuracy: 0.7286
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5575 - accuracy: 0.7293
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accuracy: 0.7323
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5529 - accuracy: 0.7322
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5516 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5505 - accuracy: 0.7324
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5496 - accuracy: 0.7342
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5486 - accuracy: 0.7337
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5479 - accura

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5541 - accuracy: 0.7314 - 357ms/epoch - 1ms/step
Loss: 0.5541431307792664, Accuracy: 0.7314285635948181


**Optimization Model 3**

In [45]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 60
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 15
hidden_nodes_layer4 = 5
nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = input_features, activation = "relu"))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#Third hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

#Fourth hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 60)                2640      
                                                                 
 dense_49 (Dense)            (None, 30)                1830      
                                                                 
 dense_50 (Dense)            (None, 15)                465       
                                                                 
 dense_51 (Dense)            (None, 5)                 80        
                                                                 
 dense_52 (Dense)            (None, 1)                 6         
                                                                 
Total params: 5021 (19.61 KB)
Trainable params: 5021 (19.61 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [47]:
# Train the model
nn4_fit = nn4.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6075 - accuracy: 0.7068
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5794 - accuracy: 0.7278
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5696 - accuracy: 0.7301
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5633 - accuracy: 0.7300
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5598 - accuracy: 0.7313
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5576 - accuracy: 0.7318
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7305
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5552 - accuracy: 0.7319
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5526 - accuracy: 0.7319
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5534 - accura